# Reinforcement Learning Based Cognitive Radio Project

*by Leif Skunberg, Michael Lyons, and Charles Culpepper, December, 2017*

## Table of Contents
* [Introduction](#Introduction)
* [Problem](#Problem)
* [Methods](#Methods)
* [Team](#Team)
* [Results](#Results)
* [Conclusions](#Conclusions)

## Introduction

#### Artificial Intelligence

Artificial intelligence, and the concept of a cognitive radio, have both been around for quite a while. Machines have been learning since at least 1950, with Alan Turing’s aptly named Learning Machine. But the mathematical foundations come from Bayes Theroem and the concept of least squares, which originates in the early 1800s. 
Artificial intelligence, as the name implies, is the creation of a synthetic form of intelligence. The goal is to create a machine that can learn. While we started with Turing’s simple machine, we would have a form of neural networks a couple of years later. The concept has been studied and abstracted almost constantly since then. Today, machine learning and artificial intelligence are one of the hottest trends in technology. A lot of the math that wasn’t worked out in the fifties, is readily available in 2017. The computers today are also infinitely more powerful, and can perform calculations in real time that early computers couldn’t process at all. These developments bring the possibility of a true cognitive radio closer to a realistic possibility

#### Cognitive Radio

Radio is ubiquitous in our society. Radio waves cover our globe, and provide everything from standard communication, to music in your car, to the frequencies used to provide the world with wireless internet. Radio waves are a type of electromagnetic radiation at a given range of frequencies. There are other waves in this spectrum, depending on the frequency. These include everything from x-rays and infrared, to UV and standard light. Currently, you tune a radio to a frequency, and it will pick up any communications on that channel. The stronger your radio, the more of the range it can access and transmit through. Some frequencies are off limits for civilians, and some others are used for specific services, like wi-fi. But within the standard range, most people know what it means to tune a radio to a frequency – the number on your favorite radio station, is a frequency.

In addition to simply tuning into the right frequency, there are several components to radio communications, the most obvious of which is interference. As you drive out of your local area, you may notice your radio station start to pick up another, or maybe die off into static. Either way, you’re seeing a form of interference. Ideally, if you can stay on the channels with the least interference, you’ll always have a clear line of communication.

A cognitive radio is one that can see, and over time, learn, which frequencies have the least interference at a given time, and can hop to the channel with least interference so that the communication stays clear. Basically, a cognitive radio is one that switches frequencies for you, depending on what it thinks is the clearest line of communication. At it’s most basic, it can simply decide between lines that are open, and lines that are already in use, keeping the user on the open lines as much as possible. This frequency hopping is called Frequency Hopping Spread Spectrum (FHSS).

The idea for a cognitive radio started back in 1998, where Joseph Mitola III made a proposal for it at a seminar for KTH (translates to Royal Institute of Technology in Stockholm). There is no fully working example of a live cognitive radio to date. Rather, it is an established concept that is accepted as possible, but is still a goal that the industry is actively working toward.
The latest trends in AI are what makes advances in cognitive radio possible. For a radio receiver to know which lines are open (or which have a satisfactory level of interference), it needs to know, compute, remember, and learn about the elements included therein. This necessitates some form of machine learning. As our technology develops, we are getting closer to processing the needed calculations in real time, meaning, as the communication happens.
Currently, most experiments regarding cognitive radio take place either with sample data, or with captured data from a live radio. While many have explored both in depth, no one has been able to perform the functionality as the communication happens.

#### Cognitive Radio Experiment in Reinforcement Learning

These same limitations impact our own experiment. Due to time constraints, we chose to run our experiment with sample data. The idea is that you should be able to apply reinforcement learning to FHSS using GNURadio, creating a software based cognitive radio. The intent is also to encrypt the message, and so a final version would be able to send end-to-end secure communications, over radio, on the clearest lines available, with no message deterioration. While we were unable to perform the functions on live data, we created sample data and a reinforcement learning algorithm in order to make a proof of concept. We are including RSA encryption code so as to have all the parts of the model we are exploring.

#### RSA Encryption

There are many ways to encrypt and decrypt data for preparation to transmit it in a radio signal. We chose an asymmetric form named RSA. The basic functionality of an asymmetric cryptographic system is the existence of two keys, a public key and a private key. It makes more intuitive sense if you consider the public key a lock. Anyone can lock something with it, but only the owner of the private key can unlock those locks. 
A user would typically use the public key of their planned recipient to encrypt some data.That data can then be encapsulated in another encryption, created by the user’s own public key. Anyone can unlock the outer layer of encryption because it uses a public key, but the inner one still needs the corresponding private key. The point of this is to prove that the sender is authentic. That user’s public key, while public, is unique, and proves that it was encrypted by the original sender. The recipient then receives the communication, uses the public key to decrypt the outer layer, and his own private key to decrypt the inner layer, and get the data. That ensures that the sender is authentic, the message is secure, and the receiver is the intended person. This system provides end to end security for any data that can be stored and transmitted digitally.


## Problem

![problem figure](problem.png)

leif

## Methods

### The Algorithm

Our reinforcement learning algorithm follows roughly the same structure as the algorithm outlined in the lecture notes.
	
	for transmission time
		next_time = time + 1
		if next_time < transmission time
			initialize_Q_table(time)
			move = get_move(time)
			update_reinforcement(time)

The key difference between a reinforcement algorithm for a maze or a game like tic-tac-toe and our simulated radio transmission is that reinforcement happens at a terminal state, like reaching the end of the maze or ending a game of tic-tac-toe. However, in our simulated radio transmission, where the move from the current state to the next state is independent, reinforcement has to happen at every hop. Our reinforcement function is simple:

if  move is optimal
	Q(state, move) = Q(state, move) * (1 + (1 - [learning rate]))
else 
	Q(state, move) = Q(state, move) * [learning rate]

In other words, if a move is optimal (it has the lowest interference), then it is increased by, say, 5%. If it is not optimal, then it is decreased by the learning rate. Ideally, based upon the assumption that there would be a strong correlation between datasets generated from a pattern, the algorithm could exploit historical knowledge about previous datasets to quickly hop frequencies.

### The Original Model

Our original model has two entities, the template and the data-set. 

The template represents a location with patterns in transmission at given times of the day. For example, a channel c might have, on average and over the long term, heavy interference at time t, while the dataset is essentially an instantiation of the template. 

This model is essentially a matrix with rows representing time snapshots and columns representing different channels. The template goes through and fills the matrix with several variables.

	alpha: percent chance to lower interference by x
	x: value to lower interference 

In addition to those two, an empty channel will have a length that propagates down the transmission matrix, with future channels in the same transmission having similar interferences. A transmission is a set of time snapshots in a column of the matrix.

When the template is instantiated into a dataset, all of the channels in all of the cells of the matrix have their interference set. A transmission’s channels throughout time will have similar interferences.

### Original Model Fails

We’ve included a discussion of the original model because it highlights our original thinking and the process, and pitfalls, of trying to create a sufficient model. 

As it turns out, the glaring problem with the original model only became apparent upon testing. After much testing, our trained Q table was only selecting optimal channels roughly 10% of the time from newly instantiated testing data sets. Confused and desperately looking for serious bugs, it finally occurred to us that we had only ten channels, and 10% is exactly what you would expect if you simply picked a channel by random! 

So what went wrong?

It turns out the original model for data generation created data sets that were too different from one another. Consider channel c1, which has an alpha of 17%. The probability that c1 will have the same value from one dataset to another is just 0.17 * 0.17 = 0.0289 or just 2.89%!

### The New Model

Our current model is much more simple. The template generates the interference values for every channel using the alpha-x system, but keeps it constant across all future datasets generated from the template. Generation of a dataset involves applying a noise function to every channel in the matrix. 

## Team

### Charles
blah blah blah

### Leif
blah blah blah

### Michael
blah blah blah

## Results

In [13]:
from Channel import *
from src.Experiment import *
from src.ReinforcementLearning import *

### Experiment
There are eight parts to an experiment as defined here:
* Transmission length: The time frame or simply the rows in the matrix.
* Channel length: The number of channels or columns in the matrix.
* Epsilon: This number controls how often a random move is selected in training. If it is 0.5, then 50% of the time a random move will be selected.
* Epsilon Decay Rate: The factor that epsilon decays by each iteration, so that it will tend towards zero. If epsilon is 80% and the epsilon decay rate is 30%, then epsilon will be 24% after one iteration.
* Training iterations: The number of times the Q table is trained on *one* data set.
* Learning rate: The rate at which reinforcements are decreased or increased.
* Train runs: The number of times the Q table is trained on *different* data sets.
* Test runs: The number of different times the Q table is trained on *different* data sets.

#### Small matrix

In [23]:
transmission_length = 10
channel_length = 5
epsilon = 0.999
epsilon_decay_rate = 0.999
training_iterations = 1000
learning_rate = 0.98
train_runs = 50
test_runs = 100

performance = run_experiment(transmission_length, channel_length, epsilon, epsilon_decay_rate, training_iterations, learning_rate, train_runs, test_runs)
print("With " + str(train_runs) + " training runs and " + str(test_runs) +" test runs, on average " + format_double(performance * 100) + "% of optimal channels were selected.")

Done Training
With 50 training runs and 100 test runs, on average 100.00% of optimal channels were selected.


#### Square Matrix

In [18]:
transmission_length = 100
channel_length = 100
epsilon = 0.999
epsilon_decay_rate = 0.999
training_iterations = 100
learning_rate = 0.98
train_runs = 50
test_runs = 100

performance = run_experiment(transmission_length, channel_length, epsilon, epsilon_decay_rate, training_iterations, learning_rate, train_runs, test_runs)
print("With " + str(train_runs) + " training runs and " + str(test_runs) +" test runs, on average " + format_double(performance * 100) + "% of optimal channels were selected.")

Done Training
With 50 training runs and 100 test runs, on average 99.78% of optimal channels were selected.


#### Long Matrix

In [22]:
transmission_length = 1000
channel_length = 10
epsilon = 0.999
epsilon_decay_rate = 0.999
training_iterations = 100
learning_rate = 0.98
train_runs = 5
test_runs = 100

performance = run_experiment(transmission_length, channel_length, epsilon, epsilon_decay_rate, training_iterations, learning_rate, train_runs, test_runs)
print("With " + str(train_runs) + " training runs and " + str(test_runs) +" test runs, on average " + format_double(performance * 100) + "% of optimal channels were selected.")

Done Training
With 5 training runs and 100 test runs, on average 100.00% of optimal channels were selected.


## Conclusions

points to touch on:
* difference between what we planned and what we actually did, why
* problem with modeling
* what we learned

### Encryption

### References

* [Russell and Norvig, 2014] Stuart Russell and Peter Norvig, [Artificial Intelligence: A Modern Approach](http://aima.cs.berkeley.edu/), Publisher. 2014.
* [chuck] me, fldfkafjlak